First lab Computational intelligence: Set covering

In [28]:
import numpy as np
from random import random
from functools import reduce
from queue import PriorityQueue

In [29]:
#costants for the complexity of the problem
PROBLEM_SIZE = 5
NUM_SETS = 10 #SETS == TILES to choose from
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
#the set of all sets/tiles we can choose from

In [70]:
#to check if we finished
def goal_check(state):
    #whate happens in case of no index? Add initial value
    return np.all(reduce(np.logical_or, [SETS[i] for i in state[0]], np.array([False for _ in range(PROBLEM_SIZE)])))


assert goal_check((set(range(NUM_SETS)), set())), "Problem not solvable" #sanity check => if i select all sets i must have a solution
#otherwise the problem is not solvable


In [81]:
#the quality of a solution depends on the number of selected tiles
#so for each state we compute the cost as the number of selected tiles
def Cost(state):
    return len(state[0])


#here the distance code seen at lesson
def Distance(state):
    already_covered = reduce(
        np.logical_or,
        [SETS[i] for i in state[0]],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
    if np.all(already_covered):
        return 0
    
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

In [86]:
#Path search implementation
frontier = PriorityQueue()
initial_state = (set(), set(range(NUM_SETS))) #initial state => no selected sets, all available index

#for starting we add in the frontier the initial state
frontier.put((0, initial_state))

(_, state) = frontier.get()

counter = 0
while not goal_check(state):
    counter += 1
    print(f"{counter}: selected state: {state} with cost {Cost(state)} and distance {Distance(state)}")

    for action in state[1]: #compute all the successors and add them to the queue
        new_state = (state[0] | {action}, state[1] - {action})
        #for the distance we consider both cost and the heuristic
        frontier.put((Cost(new_state) + Distance(new_state), new_state))
    
    (_, state) = frontier.get() #new state

print(f"Solve in {counter+1} step by taking sets {state[0]} with cost {Cost(state)}")

1: selected state: (set(), {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}) with cost 0 and distance 2
2: selected state: ({0}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}) with cost 1 and distance 1
3: selected state: ({2}, {0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}) with cost 1 and distance 1
4: selected state: ({6}, {0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14}) with cost 1 and distance 1
5: selected state: ({13}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}) with cost 1 and distance 1
6: selected state: ({14}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}) with cost 1 and distance 1
7: selected state: ({5}, {0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14}) with cost 1 and distance 1
8: selected state: ({12}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14}) with cost 1 and distance 1
9: selected state: ({11}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14}) with cost 1 and distance 1
10: selected state: ({1}, {0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}) with cost 1

KeyboardInterrupt: 

In [85]:
#the heuristic proposed by the professor consider the 
#part that we still need to cover and tries to compute the 
#number of sets we will need to cover it all (not considering useless sets)

#taking that idea i computed a slightly less efficent heuristic:
#after computing the already_covered area we compute for each not taken set
#the number of not covered area that it's going to cover
#if the best set is already covering everything we now that the cost is 1
#otherwise we will need at least 2 sets
def Distance(state):
    #compute already covered area
    already_covered = reduce(
        np.logical_or,
        [SETS[i] for i in state[0]],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

    to_cover = np.logical_not(already_covered)

    #compute the candidates from the not taken group
    candidates = [sum(np.logical_and(SETS[i], to_cover)) for i in state[1]]
    best = max(candidates)

    if(best == (PROBLEM_SIZE - sum(already_covered))):
        #we are finished by selecting the best one
        return 1
    else:
        #we need at least two
        return 2

'\ndef Distance(state):\n    #compute already covered area\n    already_covered = reduce(\n        np.logical_or,\n        [SETS[i] for i in state[0]],\n        np.array([False for _ in range(PROBLEM_SIZE)]),\n    )\n\n    to_cover = np.logical_not(already_covered) #what we still need to cover\n\n    #compute the candidates from the not taken group\n    candidates = sorted([np.logical_and(SETS[i], to_cover) for i in state[1]], reverse=True, key= lambda s : sum(s))\n\n    count = 1\n    res_set = candidates[0]\n    while not covered(already_covered, res_set):\n        #add a new set\n        res_set = np.logical_or(res_set, candidates[count])\n        count = count + 1\n\n    return count\n\ndef covered(s1, s2):\n    return np.all(reduce(np.logical_or, s1, s2))\n'

In [69]:
#we can compare the two with a bigger problem
PROBLEM_SIZE = 10
NUM_SETS = 15
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

#to compare: run one of the two cell containing Distance (the previous declaration will be deleted)
#then by running this cell you can create new tiles.
#then you can run the A* algorithm and compare the results
